In [19]:
import pandas as pd
df = pd.read_excel('중구_노드_링크_네트워크데이터.xlsx')
df = df[(df['노드링크 유형'] == 'NODE') &
        (df['육교'] == 0) &
        (df['횡단보도'] == 0)]
df.head()


,Unnamed: 0,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 WKT,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,...,읍면동명,고가도로,지하철네트워크,교량,터널,육교,횡단보도,"공원,녹지",건물내,행정동코드
0,0,NODE,POINT(127.00138520146373 37.568187437157185),66847,0.0,NaN,0,NaN,NaN,NaN,...,주교동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140605
1,1,NODE,POINT(127.01697814071575 37.56762896761354),127069,0.0,NaN,0,NaN,NaN,NaN,...,흥인동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
3,3,NODE,POINT(126.97646615812025 37.55427380378025),10380,0.0,NaN,0,NaN,NaN,NaN,...,남대문로5가,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140540
4,4,NODE,POINT(127.01917432263897 37.56617541509709),9255,0.0,NaN,0,NaN,NaN,NaN,...,황학동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140670
5,5,NODE,POINT(126.96329524184776 37.55797263212856),9627,0.0,NaN,0,NaN,NaN,NaN,...,중림동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140680


In [20]:
df.head()

,Unnamed: 0,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 WKT,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,...,읍면동명,고가도로,지하철네트워크,교량,터널,육교,횡단보도,"공원,녹지",건물내,행정동코드
0,0,NODE,POINT(127.00138520146373 37.568187437157185),66847,0.0,NaN,0,NaN,NaN,NaN,...,주교동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140605
1,1,NODE,POINT(127.01697814071575 37.56762896761354),127069,0.0,NaN,0,NaN,NaN,NaN,...,흥인동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
3,3,NODE,POINT(126.97646615812025 37.55427380378025),10380,0.0,NaN,0,NaN,NaN,NaN,...,남대문로5가,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140540
4,4,NODE,POINT(127.01917432263897 37.56617541509709),9255,0.0,NaN,0,NaN,NaN,NaN,...,황학동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140670
5,5,NODE,POINT(126.96329524184776 37.55797263212856),9627,0.0,NaN,0,NaN,NaN,NaN,...,중림동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140680


In [21]:
def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

## 다각형에서 평행좌표
def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

# def image_to_data_url(filename):
#     ext = filename.split('.')[-1]
#     prefix = f'data:image/{ext};base64,'
#     with open(filename, 'rb') as f:
#         img = f.read()
#     return prefix + base64.b64encode(img).decode('utf-8')

In [22]:
### geodata 변굥 buffer 생성기존 24시간 가동 AED와의 유효거리 기반 수요량 b(buffer_100)
import geopandas as gpd
gs = gpd.GeoSeries.from_wkt(df['노드 WKT'])
## 좌표계 설정하는 거 같음
df_dobo_100 = gpd.GeoDataFrame(df, geometry = gs, crs = 'epsg:4326')

## 버퍼설정하는 곳인거 같음
## to_crs : 좌표계 변환
df_dobo_100 = df_dobo_100.set_geometry('geometry')
df_dobo_100['buffer_100'] = df_dobo_100.to_crs('epsg:5179').buffer(100).to_crs('epsg:4326')
df_dobo_100 = df_dobo_100.set_geometry('buffer_100')
df_dobo_100['buffer_100_coordinates'] = df_dobo_100['buffer_100'].apply(polygon_to_coordinates)

df_dobo_100['lon'] = df_dobo_100['geometry'].x
df_dobo_100['lat'] = df_dobo_100['geometry'].y


In [23]:
df['읍면동코드']

0        1114015200
1        1114016300
3        1114011800
4        1114016500
5        1114017100
            ...    
12829    1114016200
12831    1114016200
12833    1114016200
12835    1114016200
12837    1114016200
Name: 읍면동코드, Length: 5092, dtype: int64

In [24]:
df.columns

Index(['Unnamed: 0', '노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT',
       '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명',
       '읍면동코드', '읍면동명', '고가도로', '지하철네트워크', '교량', '터널', '육교', '횡단보도', '공원,녹지',
       '건물내', '행정동코드'],
      dtype='object')

In [25]:
pop = pd.read_excel('중구_인구데이터.xlsx')
pop = pop.iloc[:,:4]
pop.drop('시간대구분',inplace=True,axis=1)
pop.head()

,기준일ID,행정동코드,총생활인구수
0,20231008,11140520,6686.0271
1,20231008,11140540,7505.9797
2,20231008,11140550,9002.7286
3,20231008,11140570,6983.8529
4,20231008,11140580,7685.1042


In [26]:
len(pop['행정동코드'].value_counts())

15

In [27]:
pop.head()

,기준일ID,행정동코드,총생활인구수
0,20231008,11140520,6686.0271
1,20231008,11140540,7505.9797
2,20231008,11140550,9002.7286
3,20231008,11140570,6983.8529
4,20231008,11140580,7685.1042


In [28]:
pop['행정동코드'].unique()

array([11140520, 11140540, 11140550, 11140570, 11140580, 11140590,
       11140605, 11140615, 11140625, 11140635, 11140645, 11140650,
       11140665, 11140670, 11140680], dtype=int64)

In [29]:
df['행정동코드'].unique()

array([11140605, 11140615, 11140540, 11140670, 11140680, 11140590,
       11140570, 11140580, 11140520, 11140550], dtype=int64)

In [30]:
df_dobo_100.columns

Index(['Unnamed: 0', '노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT',
       '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명',
       '읍면동코드', '읍면동명', '고가도로', '지하철네트워크', '교량', '터널', '육교', '횡단보도', '공원,녹지',
       '건물내', '행정동코드', 'geometry', 'buffer_100', 'buffer_100_coordinates',
       'lon', 'lat'],
      dtype='object')

In [31]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20520 entries, 0 to 20519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일ID   20520 non-null  int64  
 1   행정동코드   20520 non-null  int64  
 2   총생활인구수  20520 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 481.1 KB


In [32]:
df_dobo_100.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 5092 entries, 0 to 12837
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Unnamed: 0              5092 non-null   int64   
 1   노드링크 유형                 5092 non-null   object  
 2   노드 WKT                  5092 non-null   object  
 3   노드 ID                   5092 non-null   int64   
 4   노드 유형 코드                5092 non-null   float64 
 5   링크 WKT                  0 non-null      object  
 6   링크 ID                   5092 non-null   int64   
 7   링크 유형 코드                0 non-null      float64 
 8   시작노드 ID                 0 non-null      float64 
 9   종료노드 ID                 0 non-null      float64 
 10  링크 길이                   0 non-null      float64 
 11  시군구코드                   5092 non-null   int64   
 12  시군구명                    5092 non-null   object  
 13  읍면동코드                   5092 non-null   int64   
 14  읍면동명                

In [33]:
df_dobo_100.columns

Index(['Unnamed: 0', '노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT',
       '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명',
       '읍면동코드', '읍면동명', '고가도로', '지하철네트워크', '교량', '터널', '육교', '횡단보도', '공원,녹지',
       '건물내', '행정동코드', 'geometry', 'buffer_100', 'buffer_100_coordinates',
       'lon', 'lat'],
      dtype='object')

In [36]:
pop.head()

,기준일ID,행정동코드,총생활인구수
0,20231008,11140520,6686.0271
1,20231008,11140540,7505.9797
2,20231008,11140550,9002.7286
3,20231008,11140570,6983.8529
4,20231008,11140580,7685.1042


In [39]:
df_dobo_100 = pd.merge(df_dobo_100, pop[['행정동코드','총생활인구수']], how= 'left', right_on = '행정동코드', left_on = '행정동코드')

# df_dobo_100.drop(['행정동코드명','노드 WKT'],inplace=True)


Index(['Unnamed: 0', '노드링크 유형', '노드 WKT', '노드 ID', '노드 유형 코드', '링크 WKT',
       '링크 ID', '링크 유형 코드', '시작노드 ID', '종료노드 ID', '링크 길이', '시군구코드', '시군구명',
       '읍면동코드', '읍면동명', '고가도로', '지하철네트워크', '교량', '터널', '육교', '횡단보도', '공원,녹지',
       '건물내', '행정동코드', 'geometry', 'buffer_100', 'buffer_100_coordinates',
       'lon', 'lat', '총생활인구수'],
      dtype='object')

In [40]:
df_dobo_100['행정동코드'].unique()

array([11140605, 11140615, 11140540, 11140670, 11140680, 11140590,
       11140570, 11140580, 11140520, 11140550], dtype=int64)

In [42]:
df_dobo_100.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6965856 entries, 0 to 6965855
Data columns (total 30 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   Unnamed: 0              int64   
 1   노드링크 유형                 object  
 2   노드 WKT                  object  
 3   노드 ID                   int64   
 4   노드 유형 코드                float64 
 5   링크 WKT                  object  
 6   링크 ID                   int64   
 7   링크 유형 코드                float64 
 8   시작노드 ID                 float64 
 9   종료노드 ID                 float64 
 10  링크 길이                   float64 
 11  시군구코드                   int64   
 12  시군구명                    object  
 13  읍면동코드                   int64   
 14  읍면동명                    object  
 15  고가도로                    float64 
 16  지하철네트워크                 float64 
 17  교량                      float64 
 18  터널                      float64 
 19  육교                      int64   
 20  횡단보도                    int64   
 21  

In [44]:
df

,Unnamed: 0,노드링크 유형,노드 WKT,노드 ID,노드 유형 코드,링크 WKT,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,...,읍면동명,고가도로,지하철네트워크,교량,터널,육교,횡단보도,"공원,녹지",건물내,행정동코드
0,0,NODE,POINT(127.00138520146373 37.568187437157185),66847,0.0,NaN,0,NaN,NaN,NaN,...,주교동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140605
1,1,NODE,POINT(127.01697814071575 37.56762896761354),127069,0.0,NaN,0,NaN,NaN,NaN,...,흥인동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
3,3,NODE,POINT(126.97646615812025 37.55427380378025),10380,0.0,NaN,0,NaN,NaN,NaN,...,남대문로5가,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140540
4,4,NODE,POINT(127.01917432263897 37.56617541509709),9255,0.0,NaN,0,NaN,NaN,NaN,...,황학동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140670
5,5,NODE,POINT(126.96329524184776 37.55797263212856),9627,0.0,NaN,0,NaN,NaN,NaN,...,중림동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12829,12864,NODE,POINT(127.00671304470401 37.555271976044445),73931,0.0,NaN,0,NaN,NaN,NaN,...,신당동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
12831,12866,NODE,POINT(127.00609787656336 37.555217084599064),73930,0.0,NaN,0,NaN,NaN,NaN,...,신당동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
12833,12868,NODE,POINT(127.00662680680709 37.555271980916025),73929,0.0,NaN,0,NaN,NaN,NaN,...,신당동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615
12835,12870,NODE,POINT(127.006744648147 37.55508202591123),73928,0.0,NaN,0,NaN,NaN,NaN,...,신당동,NaN,NaN,NaN,NaN,0,0,NaN,NaN,11140615


In [43]:
pop

,기준일ID,행정동코드,총생활인구수
0,20231008,11140520,6686.0271
1,20231008,11140540,7505.9797
2,20231008,11140550,9002.7286
3,20231008,11140570,6983.8529
4,20231008,11140580,7685.1042
...,...,...,...
20515,20230813,11140645,13055.0703
20516,20230813,11140650,8187.9303
20517,20230813,11140665,10072.9481
20518,20230813,11140670,14415.0079
